# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
cities_weather = pd.read_csv("city_weather_data.csv")

cities_weather.head()

,Unnamed: 0,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Temperature (F),Wind Speed (mph)
0,0,George Town,20,MY,1579893009,65,5.41,100.34,80.6,3.36
1,1,Yulara,60,AU,1579893060,22,-25.24,130.99,78.8,8.05
2,2,Port Maria,20,JM,1579893225,65,18.37,-76.89,82.4,10.29
3,3,Punta Arenas,90,CL,1579893133,87,-53.15,-70.92,44.6,10.29
4,4,Ushuaia,75,AR,1579892852,54,-54.80,-68.30,55.4,3.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Lat/Lng as location, humidity for weight
locations = cities_weather[["Latitude", "Longitude"]]
humidity = cities_weather["Humidity (%)"].astype(float)

#weight
max_humidity = humidity.max()

#Figure for heatmap
fig=gmaps.figure()

#Heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max_humidity, 
                                    point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#new df updated with ideal weather conditions (Temp between 70-80 degrees F, winds < 10 mph, clear skies)

ideal_weather_condit = pd.DataFrame(cities_weather, columns = ["City", "Cloudiness (%)", "Country", "Date", "Humidity (%)", 
                                                            "Latitude", "Longitude", "Temperature (F)", "Wind Speed (mph)"])

ideal_temperature = (cities_weather["Temperature (F)"] <= 80) & (cities_weather["Temperature (F)"] > 70)
ideal_wind = cities_weather["Wind Speed (mph)"] < 10
cloudless_condits = cities_weather["Cloudiness (%)"] < 5

ideal_weather_condit = ideal_weather_condit[ideal_temperature & ideal_wind & cloudless_condits]
ideal_weather_condit

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Temperature (F),Wind Speed (mph)
14,Hermanus,0,ZA,1579893156,86,-34.42,19.23,71.01,3.00
99,Salalah,0,OM,1579893079,78,17.02,54.09,77.00,2.24
292,Voh,0,NC,1579893261,80,-20.97,164.70,75.20,5.99
325,Dumai,4,ID,1579893266,92,1.68,101.45,74.07,3.02
347,Sorong,2,ID,1579892823,88,-0.88,131.25,72.79,1.95
378,Manthani,0,IN,1579893272,46,18.65,79.67,73.71,3.87
451,Kahului,1,US,1579893080,82,20.89,-156.47,75.00,4.70
524,Santa Barbara,1,US,1579893125,55,34.42,-119.70,72.00,3.36
582,Banda Aceh,3,ID,1579893093,95,5.56,95.32,73.81,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#Add hotel column to df

hotel_df = []
ideal_weather_condit["Hotel Name"] = ""
ideal_weather_condit

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Temperature (F),Wind Speed (mph),Hotel Name
14,Hermanus,0,ZA,1579893156,86,-34.42,19.23,71.01,3.00,
99,Salalah,0,OM,1579893079,78,17.02,54.09,77.00,2.24,
292,Voh,0,NC,1579893261,80,-20.97,164.70,75.20,5.99,
325,Dumai,4,ID,1579893266,92,1.68,101.45,74.07,3.02,
347,Sorong,2,ID,1579892823,88,-0.88,131.25,72.79,1.95,
378,Manthani,0,IN,1579893272,46,18.65,79.67,73.71,3.87,
451,Kahului,1,US,1579893080,82,20.89,-156.47,75.00,4.70,
524,Santa Barbara,1,US,1579893125,55,34.42,-119.70,72.00,3.36,
582,Banda Aceh,3,ID,1579893093,95,5.56,95.32,73.81,4.70,


In [18]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

for index, row in ideal_weather_condit.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    #parameters
    params = {
        "location": f"{lat},{lng}",
        "type": "lodging",
        "radius": 5000,
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #API request
    response = requests.get(base_url, params=params)
    json = response.json()
    
    try:
        hotel_df.append(json["results"][0]["name"])
    except:
        pass
    
#Display df with hotel names
ideal_weather_condit["Hotel Name"] = hotel_df
ideal_weather_condit

,City,Cloudiness (%),Country,Date,Humidity (%),Latitude,Longitude,Temperature (F),Wind Speed (mph),Hotel Name
14,Hermanus,0,ZA,1579893156,86,-34.42,19.23,71.01,3.00,Misty Waves Boutique Hotel
99,Salalah,0,OM,1579893079,78,17.02,54.09,77.00,2.24,HAMDAN PLAZA HOTEL
292,Voh,0,NC,1579893261,80,-20.97,164.70,75.20,5.99,Le gîte du Koniambo
325,Dumai,4,ID,1579893266,92,1.68,101.45,74.07,3.02,Grand Zuri Dumai
347,Sorong,2,ID,1579892823,88,-0.88,131.25,72.79,1.95,The Waigo Hotel
378,Manthani,0,IN,1579893272,46,18.65,79.67,73.71,3.87,Social Welfare Boys Hostel
451,Kahului,1,US,1579893080,82,20.89,-156.47,75.00,4.70,Maui Seaside Hotel
524,Santa Barbara,1,US,1579893125,55,34.42,-119.70,72.00,3.36,Kimpton Canary Hotel
582,Banda Aceh,3,ID,1579893093,95,5.56,95.32,73.81,4.70,OYO 884 Rumoh PMI Hotel


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_condit.iterrows()]
marker_locations = ideal_weather_condit[["Latitude", "Longitude"]]

In [20]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Lat/Lng as locations, humidity as weight.
locations = cities_weather[["Latitude", "Longitude"]]

humidity = cities_weather["Humidity (%)"].astype(float)
max_humidity = humidity.max()

fig = gmaps.figure()

#Heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                    dissipating=False, max_intensity=max_humidity,
                                    point_radius=1)
fig.add_layer(heatmap_layer)

# Add marker layer ontop of heat map
marker_locations = [(-34.42, 19.23),
                    (17.02, 54.09),
                    (-20.97, 164.70),
                    (1.68, 101.45),
                    (-0.88, 131.25),
                    (18.65, 79.67),
                    (20.89, -156.47),
                    (34.42, -119.70),
                    (5.56, 95.32)]


markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))